In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import os

In [5]:
thepath = os.path.join(os.getcwd(),'..')
data_X = pd.read_csv(os.path.join(thepath, "data_mec_optim/marriage_personality-traits/Xvals.csv"))
data_Y = pd.read_csv(os.path.join(thepath, "data_mec_optim/marriage_personality-traits/Yvals.csv"))
affdf = pd.read_csv(os.path.join(thepath,"data_mec_optim/marriage_personality-traits/affinitymatrix.csv"))

In [6]:
nbcar = 10
affmat = affdf.iloc[0:nbcar,1:nbcar+1].values

In [12]:
sdX = data_X.std().values
sdY = data_Y.std().values
mX = data_X.mean().values
mY = data_Y.mean().values

Xvals = ((data_X-mX)/sdX).values
Yvals = ((data_Y-mY)/sdY).values

In [14]:
nobs = Xvals.shape[0]
Phi = Xvals @ affmat @ Yvals.T
obj = Phi.flatten()

In [15]:
N = Phi.shape[0]
M = Phi.shape[1]

A1 = sparse.kron(np.array(np.repeat(1,M)),sparse.identity(N))
A2 = sparse.kron(sparse.identity(M),np.array(np.repeat(1,N)))
A = sparse.vstack([A1, A2])

p = np.repeat(1/nobs, nobs)
q = np.repeat(1/nobs, nobs)
d = np.concatenate((p,q), axis = None)

In [22]:
m=grb.Model('Marriage')
x = m.addMVar(shape=len(obj), name="x")
m.setObjective(obj @ x, grb.GRB.MAXIMIZE)
# we minus 1 for the nodes because of python's 0 indexing
rhs = d
m.addConstr(A @ x == rhs, name="Constr")
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 2316 rows, 1340964 columns and 2681928 nonzeros
Model fingerprint: 0x646da5bf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-07, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 9e-04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve time: 5.87s
Presolved: 2316 rows, 1340964 columns, 2681928 nonzeros

Ordering time: 0.04s

Barrier statistics:
 AA' NZ     : 1.341e+06
 Factor NZ  : 2.683e+06 (roughly 600 MBytes of memory)
 Factor Ops : 4.144e+09 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.79387102e-13 -5.43848092e-19  1.16e+02 7.60e+00  1.68e+00     9s
   1  -1.11574956e+02  1.69369750e+01  7.24e-01 1.07e-14  1.07e-02 

In [37]:
if m.status == grb.GRB.Status.OPTIMAL:
    solution = np.array(m.getAttr('x')).reshape(N,M)
    pi = m.getAttr('pi')

Who does man $1$ match with?

In [42]:
print('Woman', np.argwhere(solution[0,:] != 0)[0][0] + 1)

Woman 576
